In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing required libraries
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import string
from wordcloud import WordCloud,STOPWORDS

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#reading the dataset
df = pd.read_csv('/content/drive/MyDrive/DA/data/train_raw.csv')

In [ ]:
df.head()

,title,text,subject,date,label
0,Arizona GOP Poised To Ban Social Justice Clas...,"Another day, another assault on the First Amen...",News,"January 14, 2017",0
1,Trump Tells Supporters He’ll Enact A Law That...,One would think that Donald Trump would ve lea...,News,"June 22, 2017",0
2,U.S. Senate to vote on Montenegro's NATO membe...,WASHINGTON (Reuters) - The U.S. Senate will vo...,politicsNews,"March 23, 2017",1
3,Paul Craig Roberts: ‘By Cooperating with Washi...,By Paul Crag RobertsA month ago I wrote a col...,US_News,"October 26, 2016",0
4,UNREAL! CBS’S TED KOPPEL Tells Sean Hannity He...,,politics,"Mar 27, 2017",0


In [ ]:
df.loc[df['text'] == ' ']

,title,text,subject,date,label
4,UNREAL! CBS’S TED KOPPEL Tells Sean Hannity He...,,politics,"Mar 27, 2017",0
11,TWO TOP Media Leftists DESTROY The Clintons: “...,,politics,"Oct 29, 2016",0
63,YES! TED CRUZ Has A Great Message For The “Nev...,,politics,"Nov 4, 2016",0
152,SECURITY ANALYST ON MIKE FLYNN PHONE CALL LEAK...,,Government News,"Feb 14, 2017",0
214,NANCY PELOSI Tries to Lead Democrats in Chants...,,politics,"Mar 24, 2017",0
...,...,...,...,...,...
30960,A MUST SEE! THE BALTIMORE RIOT STORY IN ONE BI...,,politics,"May 10, 2015",0
31249,BREAKING: DONALD TRUMP Video Statement On Leak...,,politics,"Oct 8, 2016",0
31410,LAWYER FOR ILLEGAL ALIEN RAPISTS: “Hysteria” O...,,left-news,"Mar 25, 2017",0
31420,YES! NEWT GINGRICH Rips Into ABC Anchor Over A...,,left-news,"Oct 16, 2016",0


In [ ]:
df.loc[df['title'] == ' ']

,title,text,subject,date,label


In [ ]:
#removing rows with empty text in text field
ind = df[df['text'] == ' '].index
df = df.drop(ind)

In [ ]:
df.head()

,title,text,subject,date,label
0,Arizona GOP Poised To Ban Social Justice Clas...,"Another day, another assault on the First Amen...",News,"January 14, 2017",0
1,Trump Tells Supporters He’ll Enact A Law That...,One would think that Donald Trump would ve lea...,News,"June 22, 2017",0
2,U.S. Senate to vote on Montenegro's NATO membe...,WASHINGTON (Reuters) - The U.S. Senate will vo...,politicsNews,"March 23, 2017",1
3,Paul Craig Roberts: ‘By Cooperating with Washi...,By Paul Crag RobertsA month ago I wrote a col...,US_News,"October 26, 2016",0
5,PRESIDENT TRUMP TWEETS The Perfect New Years M...,"As 2017 comes to a close, we d like to acknowl...",politics,"Dec 31, 2017",0


In [ ]:
#checking if there are any null values in the dataset
df.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [ ]:
#checking for duplicated rows in the dataset
df.duplicated().sum()

99

In [ ]:
#removing duplicates
df = df.drop_duplicates()
df.duplicated().sum()

0

In [ ]:
#checking for inconsistent values
lst = []

for i in df['date']:
  i = i[-5:]
  i = i.strip()
  lst.append(i)

df['year'] = lst
df['year'].value_counts()

2017     17823
2016     11320
2015      1718
eb-18       21
t.jpg        3
d.jpg        1
-pie/        1
Name: year, dtype: int64

In [ ]:
#removing inconsistent values
df = df[df.year!="eb-18"]
df = df[df.year!="d.jpg"]
df = df[df.year!="t.jpg"]
df = df[df.year!="ideo]"]
df = df[df.year!="dier/"]
df = df[df.year!="left/"]
df = df[df.year!="-pie/"]
df['year'].value_counts()

2017    17823
2016    11320
2015     1718
Name: year, dtype: int64

In [ ]:
#cleaning the categorical features
def cleaning(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

df['title'] = df['title'].apply(lambda x:cleaning(x))
df['text'] = df['text'].apply(lambda x:cleaning(x))

In [ ]:
#removing stopwords
stop = stopwords.words('english')
df['title'] = df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
df.head()

,title,text,subject,date,label,year
0,arizona gop poised ban social justice classes ...,another day another assault first amendment fr...,News,"January 14, 2017",0,2017
1,trump tells supporters he’ll enact law actuall...,one would think donald trump would learned les...,News,"June 22, 2017",0,2017
2,us senate vote montenegros nato membership,washington reuters us senate vote next week ra...,politicsNews,"March 23, 2017",1,2017
3,paul craig roberts ‘by cooperating washington ...,paul crag robertsa month ago wrote column hesi...,US_News,"October 26, 2016",0,2016
5,president trump tweets perfect new years messa...,comes close like acknowledge gratitude preside...,politics,"Dec 31, 2017",0,2017


In [ ]:
df.to_csv('/content/drive/MyDrive/DA/data/train.csv', index = False)